# Network analysis - World of Warcraft

#### Import API Key

In [1]:
with open('battle_net_api.key', 'r') as f:
    key = f.read()

### Get data from Battle.net

In [2]:
import battlenet_api
reload(battlenet_api)

<module 'battlenet_api' from 'battlenet_api.pyc'>

In [3]:
bn = battlenet_api.BattleNet(public_key=key)

#### Get data for a PVE Realm

In [4]:
realms = bn.realms()

In [5]:
print "Number of Realms: %d" % len(realms)

Number of Realms: 246


In [6]:
realms_pve = [r for r in realms if r['type']=='pve']

In [7]:
print "Number of PVE Realms: %d" % len(realms_pve)

Number of PVE Realms: 117


In [8]:
realm = realms_pve[0]

In [11]:
print "Selected Realm: %s" % realm['name']

Selected Realm: Aerie Peak


#### Get data for Challenges

In [12]:
challenges = bn.challenges(realm['name'])

In [13]:
print "Number of Challenges: %d" % len(challenges)

Number of Challenges: 17


In [15]:
groups = challenges[0]['groups']

In [16]:
print "Number of groups per Challenge: %d" % len(groups)

Number of groups per Challenge: 100


#### Distribution of number of members in groups

In [17]:
import numpy as np

In [18]:
names = []
for challenge in challenges:
    num_members = []  # list of num. of members
    for group in challenge['groups']:
        members = group['members']
        num_members += [len(members)]
        for m in members:
            if m.has_key('character'):
                 names += [m['character']['name']]
#         print ', '.join(names)

    print "Challenge %s group members: %0.2f average and %0.2f std" % \
    (challenge['map']['name'], np.mean(num_members), np.std(num_members))

Challenge Auchindoun group members: 5.00 average and 0.00 std
Challenge Bloodmaul Slag Mines group members: 5.00 average and 0.00 std
Challenge Gate of the Setting Sun group members: 5.00 average and 0.00 std
Challenge Grimrail Depot group members: 5.00 average and 0.00 std
Challenge Iron Docks group members: 5.00 average and 0.00 std
Challenge Mogu'shan Palace group members: 5.00 average and 0.00 std
Challenge Scarlet Halls group members: 5.00 average and 0.00 std
Challenge Scarlet Monastery group members: 5.00 average and 0.00 std
Challenge Scholomance group members: 5.00 average and 0.00 std
Challenge Shado-Pan Monastery group members: 5.00 average and 0.00 std
Challenge Shadowmoon Burial Grounds group members: 5.00 average and 0.00 std
Challenge Siege of Niuzao Temple group members: 5.00 average and 0.00 std
Challenge Skyreach group members: 5.00 average and 0.00 std
Challenge Stormstout Brewery group members: 5.00 average and 0.00 std
Challenge Temple of the Jade Serpent group mem

In [19]:
print "Number of unique characters: %d" % len(set(names))

Number of unique characters: 1057


#### Create a network of characters

In [20]:
import networkx as nx

In [21]:
import algos
reload(algos)

<module 'algos' from 'algos.pyc'>

In [22]:
# Create a graph of all characters as nodes 
# and number of challanges on leaderboard as edges
g = nx.Graph()

for challenge in challenges:
    for group in challenge['groups']:
        names = []  # List of node names to add to graph
        for member in group['members']:
            if member.has_key('character'):  # Only add if character info is available
                name = member['character']['name']
#                 print name
                names += [name]
#         print names
        g = algos.add_clique_with_weights(g, names)

#### Calculate correlation between algebraic connectivity and performance

In [23]:
from scipy.stats import pearsonr

In [24]:
ac = []
time = []

for challenge in challenges:
    for group in challenge['groups']:
        names = []  # List of node names to add to graph
        for member in group['members']:
            if member.has_key('character'):  # Only add if character info is available
                name = member['character']['name']
                names += [name]
        
        g_team = g.subgraph(names)
        if len(g_team.nodes()) >1:
            ac += [nx.algebraic_connectivity(g_team)]
            time += [group['time']['time']]

In [25]:
ac = np.array(ac)
time = np.array(time)
r, p = pearsonr(ac, time)

In [26]:
print "Algebraic Connectivity vs. challenge completion time: Pearson coeff = %0.2f, p-value = %0.6f" % (r, p)

Algebraic Connectivity vs. challenge completion time: Pearson coeff = -0.12, p-value = 0.000002
